In [2]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from pydantic import BaseModel
import json
from typing import List
import rich

In [3]:
load_dotenv()

api_key = os.getenv('OPENAI_API_KEY')
MODEL = "gpt-4o-mini"

openai = OpenAI()

Passing json schema or Pydantic object in API call to enforce structured output

# Chat Completion API

https://platform.openai.com/docs/guides/structured-outputs?api-mode=chat

To enforce structued output we will use `parse` function of API like this `openai.beta.chat.completions.parse`, and pass `response_format` property to send json schema

In [11]:
response = openai.beta.chat.completions.parse(
    model=MODEL,
    messages=[
        {"role": "developer", "content": "Extract the event information. "},
        {"role": "user", "content": "Alice and Bob are going to a science fair on Friday."},
    ],
    # response_format property is different in chat and responses api
    response_format={
        # schema format is also bit different in chat and responses api
        "type": "json_schema",
        "json_schema": {
            "name": "calendar_event",
            "schema": {
                "type": "object",
                "properties": {
                    "name": { "type": "string"},
                    "date": { "type": "string" },
                    "participants": { "type": "array", "items": { "type": "string" }},
                },
                "required": ["name", "date", "participants"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
)
print(response.choices[0].message.content)
dictionary = json.loads(response.choices[0].message.content)
rich.print(dictionary)
print(dictionary["name"])


print()
# This will return None, but if we provide pydantic's BaseModel object in API call 
# then we will get parsed object. See example below
print(response.choices[0].message.parsed) 

{"name":"Science Fair","date":"Friday","participants":["Alice","Bob"]}


{'name': 'Science Fair', 'date': 'Friday', 'participants': ['Alice', 'Bob']}

Science Fair

None


A similar example, except for the user input.

In [6]:
response = openai.beta.chat.completions.parse(
    model=MODEL,
    messages=[
        {"role": "developer", "content": "Extract the event information. "},
        {"role": "user", "content": "Leonardo, Ivan and Alex will be joining Taylor for dinner on Tuesday night."},
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "calendar_event",
            "schema": {
                "type": "object",
                "properties": {
                    "name": { "type": "string"},
                    "date": { "type": "string" },
                    "participants": { "type": "array", "items": { "type": "string" }},
                },
                "required": ["name", "date", "participants"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
)
print(response.choices[0].message.content)
dictionary = json.loads(response.choices[0].message.content)
print(dictionary["name"])

{"name":"Dinner with Taylor","date":"Tuesday night","participants":["Leonardo","Ivan","Alex","Taylor"]}
Dinner with Taylor


Defining object schemas using Pydantic

Note: `response.choices[0].message.parsed` directly returns object

In [12]:
class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]
    
response = openai.beta.chat.completions.parse(
    model=MODEL,
    messages=[
        {"role": "developer", "content": "Extract the event information. "},
        {"role": "user", "content": "Leonardo, Ivan and Alex will be joining Taylor for dinner on Tuesday night."},
    ],
    response_format=CalendarEvent
)
# We have json string which we can convert into json object
print(response.choices[0].message.content)
dictionary = json.loads(response.choices[0].message.content)
print(dictionary["name"])

print()
# Also 'response.choices[0].message' and 'parsed' property that will return parsed object
print(response.choices[0].message.parsed)
rich.print(response.choices[0].message.parsed)
obj = response.choices[0].message.parsed
print(obj.name)

{"name":"Dinner with Taylor","date":"Tuesday night","participants":["Leonardo","Ivan","Alex","Taylor"]}
Dinner with Taylor

name='Dinner with Taylor' date='Tuesday night' participants=['Leonardo', 'Ivan', 'Alex', 'Taylor']


CalendarEvent(name='Dinner with Taylor', date='Tuesday night', participants=['Leonardo', 'Ivan', 'Alex', 'Taylor'])

Dinner with Taylor


# Responses API

https://platform.openai.com/docs/guides/structured-outputs?api-mode=responses

To enforce structued output we can use same `create` function and pass `text` property to send json schema

In [15]:
response = openai.responses.create(
    model=MODEL,
    input=[
        {"role": "developer", "content": "Extract the event information. "},
        {"role": "user", "content": "Leonardo, Ivan and Alex will be joining Taylor for dinner on Tuesday night."},
    ],
    # text property is different in chat and responses api
    text={
        # schema format is also bit different in chat and responses api
        "format": {
            "type": "json_schema",
            "name": "calendar_event",
            "schema": {
                "type": "object",
                "properties": {
                    "name": { "type": "string"},
                    "date": { "type": "string" },
                    "participants": { "type": "array", "items": { "type": "string" }},
                },
                "required": ["name", "date", "participants"],
                "additionalProperties": False
            },
            "strict": True
        }
    }
)
print(response.output_text)
dictionary = json.loads(response.output_text)
print(dictionary["name"])

{"name":"Dinner with Taylor","date":"Tuesday night","participants":["Leonardo","Ivan","Alex","Taylor"]}
Dinner with Taylor


In the example below, we use `openai.responses.parse()`, and the property for format is `text_format` instead of `text`.

https://github.com/openai/openai-python/blob/main/examples/responses/structured_outputs.py

In [17]:
class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]
    
response = openai.responses.parse(
    model=MODEL,
    input=[
        {"role": "developer", "content": "Extract the event information. "},
        {"role": "user", "content": "Leonardo, Ivan and Alex will be joining Taylor for dinner on Tuesday night."},
    ],
    text_format=CalendarEvent
)
print(response.output_text)
dictionary = json.loads(response.output_text)
print(dictionary["name"])

{"name":"Dinner with Taylor","date":"Tuesday night","participants":["Leonardo","Ivan","Alex","Taylor"]}
Dinner with Taylor


Similar example, execpt for handling parsed object in response

Note: `response.output[0].content[0].parsed` directly returns object

In [18]:
class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]
    
response = openai.responses.parse(
    model=MODEL,
    input=[
        {"role": "developer", "content": "Extract the event information. "},
        {"role": "user", "content": "Leonardo, Ivan and Alex will be joining Taylor for dinner on Tuesday night."},
    ],
    text_format=CalendarEvent
)
print(response.output_text)
print()
print(response)
print()
print(response.output[0].content[0].parsed)
rich.print(response.output[0].content[0].parsed)
print()
obj = response.output[0].content[0].parsed
print(obj.date)

{"name":"Dinner with Taylor","date":"Tuesday night","participants":["Leonardo","Ivan","Alex","Taylor"]}

ParsedResponse[CalendarEvent](id='resp_67e21e4e954081929f08cbb332904a9b051b9de37205a58e', created_at=1742872142.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ParsedResponseOutputMessage[CalendarEvent](id='msg_67e21e4f2eb481928b88bd6b131ddac6051b9de37205a58e', content=[ParsedResponseOutputText[CalendarEvent](annotations=[], text='{"name":"Dinner with Taylor","date":"Tuesday night","participants":["Leonardo","Ivan","Alex","Taylor"]}', type='output_text', parsed=CalendarEvent(name='Dinner with Taylor', date='Tuesday night', participants=['Leonardo', 'Ivan', 'Alex', 'Taylor']))], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, max_output_tokens=None, previous_response_id=None, reasoning=Reasoning(effort=None, gen

CalendarEvent(name='Dinner with Taylor', date='Tuesday night', participants=['Leonardo', 'Ivan', 'Alex', 'Taylor'])


Tuesday night


You can ask the model to output an answer in a structured, step-by-step way, to guide the user through the solution.

In [19]:
class Step(BaseModel):
    explanation: str
    output: str


class MathResponse(BaseModel):
    steps: List[Step]
    final_answer: str

response = openai.responses.parse(
    model=MODEL,
    input=[
        {"role": "developer", "content": "You are a helpful math tutor. Guide the user through the solution step by step."},
        {"role": "user", "content": "how can I solve 8x + 7 = -23"},
    ],
    text_format=MathResponse
)

math_reasoning = response.output[0].content[0].parsed
print(math_reasoning)
print()
print(math_reasoning.final_answer)
print()
rich.print(math_reasoning)

steps=[Step(explanation='Start with the equation: 8x + 7 = -23.', output='8x + 7 = -23'), Step(explanation='Subtract 7 from both sides to isolate the term containing x.', output='8x + 7 - 7 = -23 - 7'), Step(explanation='This simplifies to: 8x = -30.', output='8x = -30'), Step(explanation='Now, divide both sides by 8 to solve for x.', output='x = -30 / 8'), Step(explanation='Simplify the fraction: -30 / 8 can be reduced to -15 / 4.', output='x = -15/4')] final_answer='x = -15/4'

x = -15/4



MathResponse(
    steps=[
        Step(explanation='Start with the equation: 8x + 7 = -23.', output='8x + 7 = -23'),
        Step(
            explanation='Subtract 7 from both sides to isolate the term containing x.',
            output='8x + 7 - 7 = -23 - 7'
        ),
        Step(explanation='This simplifies to: 8x = -30.', output='8x = -30'),
        Step(explanation='Now, divide both sides by 8 to solve for x.', output='x = -30 / 8'),
        Step(explanation='Simplify the fraction: -30 / 8 can be reduced to -15 / 4.', output='x = -15/4')
    ],
    final_answer='x = -15/4'
)